# <span style="font-family:Papyrus; font-size:4em;">Title</span>

## Introduction  
1). background information  
2). the question we will ask (one and one only)  
3). identify and describe the **data set** we will use to answer the quetsion  

## Preliminary exploratory data analysis  
read  
wrangle  
summarize(into at least one table)   
visualize (at least one plot)  

## Method  
1). explain how we will conduct data analysis  
2). describe how we will visualize result

## Expected outcomes and significance  